<a href="https://colab.research.google.com/github/Christian-F-Badillo/Profesor_Resume/blob/master/Review_Prof_Sentimental_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cuadernillo para el Análisis de Sentimientos

> Autor: Christian Badillo

* Se realiza un análisis de sentimientos para clasificar las evaluaciones realizadas por los alumnos a los profesores en una escala de 5 estrellas usando un modelo pre-entrenado alojado en Hugging Face

- Se recomienda el uso de GPU gratuita de Google Colab para replicar el análisis de manera eficiente.

In [5]:
import pandas as pd

In [6]:
reviews_raw = pd.read_csv('https://raw.githubusercontent.com/Christian-F-Badillo/Profesor_Resume/refs/heads/master/data/merged_reviews.csv?token=GHSAT0AAAAAADHFDFWZFKOSAGHRGDOUHLFS2EC2ADQ')

In [7]:
reviews_raw.head(10)

,profesor,fecha,comentario,calidad_general,facilidad
0,"Abrego González, Leticia",14/May/2021,NaN,2.0,2.0
1,"Abrego González, Leticia",16/Ene/2017,NaN,2.0,2.0
2,"Abrego González, Leticia",31/Jul/2015,nunca asiste o llega tarde se la pasa comiendo...,4.0,8.0
3,"Abrego González, Leticia",09/Jul/2015,"casi nunca va, y cuando va deja tareas en equi...",6.0,10.0
4,"Abrego González, Leticia",25/Jul/2014,"Es una maestra pésima, me tocó en la práctica ...",2.0,6.0
5,"Abrego González, Leticia",08/Jul/2014,"No asiste, no le interesa que aprendas, no te ...",3.0,6.0
6,"Abrego González, Leticia",01/Jul/2014,Se nota su falta de interés. Obviamente no apr...,2.0,8.0
7,"Abrego González, Leticia",12/Jun/2014,"Es pésima, no asiste, es desinteresada, muy fl...",2.0,10.0
8,"Abrego González, Leticia",20/May/2014,[Comentario esperando revisión],10.0,10.0
9,"Abrego González, Leticia",25/Jul/2013,NaN,2.0,2.0


In [8]:
# Check for NaN's
reviews_raw.isna().sum()

,0
profesor,0
fecha,0
comentario,760
calidad_general,0
facilidad,0


In [9]:
# Remove NaN's
reviews_raw.dropna(inplace=True)

In [10]:
# Check for NaN's
reviews_raw.isna().sum()

,0
profesor,0
fecha,0
comentario,0
calidad_general,0
facilidad,0


In [11]:
reviews_raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12302 entries, 2 to 13061
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   profesor         12302 non-null  object 
 1   fecha            12302 non-null  object 
 2   comentario       12302 non-null  object 
 3   calidad_general  12302 non-null  float64
 4   facilidad        12302 non-null  float64
dtypes: float64(2), object(3)
memory usage: 576.7+ KB


In [12]:
reviews = reviews_raw.copy().reset_index(drop=True)

## Sentimental Analysis

In [13]:
rev_text = reviews['comentario']

In [14]:
rev_text.head(10)

,comentario
0,nunca asiste o llega tarde se la pasa comiendo...
1,"casi nunca va, y cuando va deja tareas en equi..."
2,"Es una maestra pésima, me tocó en la práctica ..."
3,"No asiste, no le interesa que aprendas, no te ..."
4,Se nota su falta de interés. Obviamente no apr...
5,"Es pésima, no asiste, es desinteresada, muy fl..."
6,[Comentario esperando revisión]
7,"No asiste, no le interesa que aprendas, no te ..."
8,Se nota su falta de interés. Obviamente no apr...
9,"Es pésima, no asiste, es desinteresada, muy fl..."


In [15]:
print("Total de Comentarios: ", rev_text.shape[0])

Total de Comentarios:  12302


In [16]:
# Remove invalid text review, id est: "[Comentario esperando revisión]"
rev_text = rev_text[rev_text != '[Comentario esperando revisión]']

In [17]:
print("Total de Comentarios: ", rev_text.shape[0])

Total de Comentarios:  11261


### Model for Sentimental Analysis

- I use the [bert-base-multilingual-uncased-sentiment](https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment) model from [Hugging Face](https://huggingface.co/) to clasify the reviews in a five-stars format.

In [18]:
# Install the API to Hugging Face
!pip install -q transformers

In [19]:
from transformers import pipeline

In [21]:
import torch
device = 0 if torch.cuda.is_available() else -1
print(f"Device: {device}")
classifier = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment", device=device)

Device: 0


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


In [22]:
from tqdm import tqdm

tqdm.pandas()  # barra de progreso para apply

def clasificar_batch(lista_textos):
    return [classifier(texto)[0]['label'] for texto in lista_textos]

# Divide en lotes (batching manual)
batch_size = 128
sentimientos = []

for i in tqdm(range(0, len(rev_text), batch_size)):
    batch = rev_text.iloc[i:i+batch_size].tolist()
    resultados = clasificar_batch(batch)
    sentimientos.extend(resultados)

rev_text['sentimiento'] = sentimientos

100%|██████████| 88/88 [02:06<00:00,  1.43s/it]


In [29]:
len(sentimientos)

11261

In [34]:
len(rev_text)

11261

In [35]:
revs = {
    'review': rev_text,
    'rating': sentimientos
}
reviews_sent_analysis = pd.DataFrame(revs)

# Merge Data

In [36]:
reviews.shape

(12302, 5)

In [37]:
reviews = reviews[reviews['comentario'] != '[Comentario esperando revisión]']

In [38]:
final_data = pd.merge(reviews, reviews_sent_analysis, left_on='comentario', right_on='review', how='left')

In [39]:
final_data.head()

,profesor,fecha,comentario,calidad_general,facilidad,review,rating
0,"Abrego González, Leticia",31/Jul/2015,nunca asiste o llega tarde se la pasa comiendo...,4.0,8.0,nunca asiste o llega tarde se la pasa comiendo...,1 star
1,"Abrego González, Leticia",09/Jul/2015,"casi nunca va, y cuando va deja tareas en equi...",6.0,10.0,"casi nunca va, y cuando va deja tareas en equi...",1 star
2,"Abrego González, Leticia",25/Jul/2014,"Es una maestra pésima, me tocó en la práctica ...",2.0,6.0,"Es una maestra pésima, me tocó en la práctica ...",1 star
3,"Abrego González, Leticia",08/Jul/2014,"No asiste, no le interesa que aprendas, no te ...",3.0,6.0,"No asiste, no le interesa que aprendas, no te ...",1 star
4,"Abrego González, Leticia",08/Jul/2014,"No asiste, no le interesa que aprendas, no te ...",3.0,6.0,"No asiste, no le interesa que aprendas, no te ...",1 star


In [40]:
final_data.isna().sum()

,0
profesor,0
fecha,0
comentario,0
calidad_general,0
facilidad,0
review,0
rating,0


In [43]:
final_data.rating.value_counts()

,count
rating,
5 stars,4327
4 stars,4055
3 stars,2236
2 stars,1861
1 star,1718


In [41]:
final_data.to_csv('reviews_plus_score.csv', index=False)